In [1]:
original_prica = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [2]:
from collections import defaultdict

In [3]:
price = defaultdict(int)

In [4]:
for i,p in enumerate(original_prica):
    price[i+1] = p

In [5]:
price[10]

30

## Get the max splitting by enumerate

In [6]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30})

In [7]:
def r(n):
    
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )

In [8]:
r(10)

30

In [9]:
import time

In [10]:
%%time
r(15)

Wall time: 6.98 s


43

In [11]:
import time

In [12]:
#@get_time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [13]:
start = time.time()
print(fibonacci(34))
end = time.time()
print(end-start)

5702887
2.164656639099121


In [14]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [15]:
start = time.time()
print(fibonacci_op(64))
end = time.time()
print(end-start)

14662949395604
0.00099945068359375


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [16]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [17]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [18]:
def f():
    print('HELLO')

In [19]:
f = f1(f)

In [20]:
print(f.__name__)

wrapper


In [21]:
@f1
def g(a):
    print(a)

In [22]:
g('hello')

Started
hello
Ended


In [23]:
def k(*arg,**kwargs):
    print(kwargs)

In [24]:
k(6,b=5)

{'b': 5}


In [25]:
from functools import wraps

In [26]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [27]:
solution = {}

In [28]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [29]:
r(15)

43

In [30]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             11: 0})

In [31]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (10, 1),
 12: (10, 2),
 13: (10, 3),
 14: (12, 2),
 15: (13, 2)}

# How do we parse solution?¶

In [32]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [33]:
r(15)

43

In [34]:
parse_solution(15)

[10, 3, 2]

# Edit Distance

In [35]:
solution = {}

In [36]:
from functools import lru_cache

In [37]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: 
        return len(string2)
    if len(string2) == 0: 
        return len(string1)
    #print(string1, string2)
    #print(len(string1), len(string2))
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    #print(candidates)
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [38]:
s1 = 'ADCDEG'
s2 = 'ABCC'
edit_distance(s1, s2)

4

In [39]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('AD', 'A'): 'DEL D',
 ('AD', 'AB'): 'SUB D => B',
 ('AD', 'ABC'): 'ADD C',
 ('AD', 'ABCC'): 'ADD C',
 ('ADC', 'A'): 'DEL C',
 ('ADC', 'AB'): 'DEL C',
 ('ADC', 'ABC'): '',
 ('ADC', 'ABCC'): 'ADD C',
 ('ADCD', 'A'): 'DEL D',
 ('ADCD', 'AB'): 'DEL D',
 ('ADCD', 'ABC'): 'DEL D',
 ('ADCD', 'ABCC'): 'SUB D => C',
 ('ADCDE', 'A'): 'DEL E',
 ('ADCDE', 'AB'): 'DEL E',
 ('ADCDE', 'ABC'): 'DEL E',
 ('ADCDE', 'ABCC'): 'DEL E',
 ('ADCDEG', 'A'): 'DEL G',
 ('ADCDEG', 'AB'): 'DEL G',
 ('ADCDEG', 'ABC'): 'DEL G',
 ('ADCDEG', 'ABCC'): 'DEL G'}

In [40]:
def parse_solution(s1, s2):
    result = {}
    if len(s1) >= len(s2):
        j = len(s1) - len(s2)
        for i in range(len(s2)):
            result[(s1[:i+1], s2[:i+1])] = solution[(s1[:i+1], s2[:i+1])]
        for z in range(j):
            result[(s1[:len(s2)+z+1], s2)] = solution[(s1[:len(s2)+z+1], s2)]
    elif len(s1) < len(s2):
        j = len(s2) - len(s1)
        for i in range(len(s1)):
            result[(s1[:i+1], s2[:i+1])] = solution[(s1[:i+1], s2[:i+1])]
        for z in range(j):
            result[(s1, s2[:len(s2)+z+1])] = solution[(s1, s2[:len(s2)+z+1])]
    for key in list(result.keys()):
        if not result[key]:
            del result[key]
    return result
print(parse_solution(s1, s2))

{('AD', 'AB'): 'SUB D => B', ('ADCD', 'ABCC'): 'SUB D => C', ('ADCDE', 'ABCC'): 'DEL E', ('ADCDEG', 'ABCC'): 'DEL G'}


## Todo: Parse Solution is our homework

# Problem Case 3: Pinyin Auto Correction Problem

In [25]:
chinese_dataset = 'data/article_9k.txt'

In [26]:
CHINESE_CHARACTERS = open(chinese_dataset).read()

In [27]:
CHINESE_CHARACTERS[:100]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MI'

In [28]:
import pinyin

In [29]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [30]:
def chinese_to_pinyin(character):
    
    return pinyin.get(character, format='strip', delimiter=' ')

In [31]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARACTERS)

In [32]:
import re

In [33]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [34]:
CHINESE_CHARATERS_COPYS[:1000]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi zan ting geng xin fa bu han kai fa ban ti yan ban nei ce wen ding ban zan bu shou ying xiang yi que bao gong cheng shi ke yi ji zhong quan bu jing li jin xing xi tong you hua gong zuo you ren cai ce zhe ye shi jiang jing li zhu yao yong dao M I U I 9 de yan fa zhi zhong M I U I 8 qu nian 5 yue fa bu ju jin yi you yi nian you yu ye shi shi hou geng xin huan dai le dang ran guan yu M I U I 9 de que qie xin xi wo men huan shi deng dai guan fang xiao xi \n xiao long 8 3 5 zuo wei wei yi tong guo W i n d o w s 1 0 zhuo mian ping tai ren zheng de A R M chu li qi gao tong qiang diao bu hui yin wei zhi kao lv xing neng er qu ping bi diao xiao he xin xiang fan ta men zheng lian shou wei ruan zhao dao yi zhong shi he zhuo mian ping tai de jian gu xing neng he gong hao de wan mei fang an bao dao cheng wei ruan yi jing na dao le yi xie xin de yuan ma yi bian W i n d o w s 1 0 geng hao di li jie b i

In [35]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [36]:
CHINESE_CHARATERS_COPYS = [pinyin for pinyin in tokens(CHINESE_CHARATERS_COPYS) if len(pinyin) > 1]

In [19]:
from collections import Counter, defaultdict

In [37]:
PINYIN_COUNT = Counter(CHINESE_CHARATERS_COPYS)

In [171]:
PINYIN_COUNT.get

<function Counter.get(key, default=None, /)>

In [172]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    
    return max(candidates,key=PINYIN_COUNT.get)

In [339]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [340]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [341]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [342]:
print(edits0('pinyin'))

{'pinyin'}


In [346]:
print(len(edits1('pinyin')))

338


# Test

In [178]:
correct('ping')

'ping'

In [179]:
correct('ynig')

'ying'

In [184]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [185]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [186]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

# 这道题脑袋短路，在拼接字符串上有限时间内无法完成，请老师指教。

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

In [327]:
#因为中文拼音最多5个字母，所有这里最长切割为7个字母
def cut(s: str):
    results = []
    num = 0
    # x + 1 表示子字符串长度
    for x in range(7):
        # i 表示偏移量
        for i in range(len(s) - x):
            results.append(s[i:i + x + 1])
    return results

cut_words = cut('woyaoshangqinghua')
len(cut_words), cut_words[-10: ]

(98,
 ['oyaosha',
  'yaoshan',
  'aoshang',
  'oshangq',
  'shangqi',
  'hangqin',
  'angqing',
  'ngqingh',
  'gqinghu',
  'qinghua'])

In [313]:
memory = []

In [337]:
from itertools import combinations
print(list(combinations(cut_words, 4)))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [328]:
len('woyaoshangqinghua')

17

In [289]:
random_rank(cut_words)

w
o
y
a
o
s
h
a
n
g
q
i
n
g
h
u
a
wo
oy
ya
ao
os
sh
ha
an
ng
gq
qi
in
ng
gh
hu
ua
woy
oya
yao
aos
osh
sha
han
ang
ngq
gqi
qin
ing
ngh
ghu
hua
woya
oyao
yaos
aosh
osha
shan
hang
angq
ngqi
gqin
qing
ingh
nghu
ghua
woyao
oyaos
yaosh
aosha
oshan
shang
hangq
angqi
ngqin
gqing
qingh
inghu
nghua
woyaos
oyaosh
yaosha
aoshan
oshang
shangq
hangqi
angqin
ngqing
gqingh
qinghu
inghua
woyaosh
oyaosha
yaoshan
aoshang
oshangq
shangqi
hangqin
angqing
ngqingh
gqinghu
qinghua



''